In [3]:
pip install selenium

     |████████████████████████████████| 904 kB 1.6 MB/s eta 0:00:01
Note: you may need to restart the kernel to use updated packages.


In [64]:
# importing packages
import pandas as pd
import re


from bs4 import BeautifulSoup as bs
from datetime import date, timedelta, datetime
from IPython.core.display import clear_output
from random import randint
import requests
from selenium import webdriver
from selenium.webdriver.common.action_chains import ActionChains
from time import sleep
from time import time
start_time = time()

from warnings import warn

In [172]:
# replace variables here.
linkedin_url = "https://www.linkedin.com/jobs/search/?geoId=105149562&keywords=data%20analyst&location=South%20Korea&sortBY=DD"
no_of_jobs = 86

In [173]:
requests.get(linkedin_url)

<Response [200]>

In [175]:
# this will open up new window with the url provided above 
driver = webdriver.Chrome()
driver.get(linkedin_url)
sleep(3)
action = ActionChains(driver)

In [176]:
source1 = bs(driver.page_source)
type(source1)

bs4.BeautifulSoup

In [178]:

# to show more jobs. Depends on number of jobs selected
i = 2
while i <= (no_of_jobs/86): 
    driver.find_element_by_xpath('/html/body/main/div/section/button').click()
    i = i + 1
    sleep(5)

In [179]:
# parsing the visible webpage
pageSource = driver.page_source
lxml_soup = BeautifulSoup(pageSource, 'lxml')

# searching for all job containers
job_container = lxml_soup.find('ul', class_ = 'jobs-search__results-list')

print('You are scraping information about {} jobs.'.format(len(job_container)))

You are scraping information about 23 jobs.


In [184]:
job_container

<ul class="jobs-search__results-list"><li class="result-card job-result-card result-card--with-hover-state job-card__contents--active" data-column="1" data-entity-urn="urn:li:jobPosting:2401687696" data-id="2401687696" data-row="1" data-search-id="75622488-8146-43e8-bcc6-18aed8ee241d" data-tracking-id="aQpdiKGkavcxp6ezJdJ1lA=="><a class="result-card__full-card-link" data-tracking-client-ingraph="" data-tracking-control-name="public_jobs_job-result-card_result-card_full-click" data-tracking-will-navigate="" href="https://kr.linkedin.com/jobs/view/digital-optimisation-analyst-at-dyson-2401687696?refId=75622488-8146-43e8-bcc6-18aed8ee241d&amp;trackingId=aQpdiKGkavcxp6ezJdJ1lA%3D%3D&amp;position=1&amp;pageNum=0&amp;trk=public_jobs_job-result-card_result-card_full-click"><span class="screen-reader-text">Digital Optimisation Analyst</span></a><img alt="Dyson" aria-hidden="true" class="artdeco-entity-image job-result-card artdeco-entity-image--square-4 job-result-card--square-4 result-card__i

In [185]:
# setting up list for job information
job_id = []
post_title = []
company_name = []
post_date = []
job_location = []
job_desc = []
level = []
emp_type = []
functions = []
industries = []

# for loop for job title, company, id, location and date posted
for job in job_container:
    
    # job title
    job_titles = job.find("span", class_="screen-reader-text").text
    post_title.append(job_titles)
    
    # linkedin job id
    job_ids = job.find('a', href=True)['href']
    job_ids = re.findall(r'(?!-)([0-9]*)(?=\?)',job_ids)[0]
    job_id.append(job_ids)
    
    # company name
    company_names = job.select_one('img')['alt']
    company_name.append(company_names)
    
    # job location
    job_locations = job.find("span", class_="job-result-card__location").text
    job_location.append(job_locations)
    
    # posting date
    post_dates = job.select_one('time')['datetime']
    post_date.append(post_dates)

# for loop for job description and criterias
for x in range(1,len(job_id)+1):
    
    # clicking on different job containers to view information about the job
    job_xpath = '/html/body/main/div/section/ul/li[{}]/img'.format(x)
    driver.find_element_by_xpath(job_xpath).click()
    sleep(3)
    
    # job description
    jobdesc_xpath = '/html/body/main/section/div[2]/section[2]/div'
    job_descs = driver.find_element_by_xpath(jobdesc_xpath).text
    job_desc.append(job_descs)
    
    # job criteria container below the description
    job_criteria_container = lxml_soup.find('ul', class_ = 'job-criteria__list')
    all_job_criterias = job_criteria_container.find_all("span", class_='job-criteria__text job-criteria__text--criteria')
    
    # Seniority level
    seniority_xpath = '/html/body/main/section/div[2]/section[2]/ul/li[1]'
    seniority = driver.find_element_by_xpath(seniority_xpath).text.splitlines(0)[1]
    level.append(seniority)
    
    # Employment type
    type_xpath = '/html/body/main/section/div[2]/section[2]/ul/li[2]'
    employment_type = driver.find_element_by_xpath(type_xpath).text.splitlines(0)[1]
    emp_type.append(employment_type)
    
    # Job function
    function_xpath = '/html/body/main/section/div[2]/section[2]/ul/li[3]'
    job_function = driver.find_element_by_xpath(function_xpath).text.splitlines(0)[1]
    functions.append(job_function)
    
    # Industries
    industry_xpath = '/html/body/main/section/div[2]/section[2]/ul/li[4]'
    industry_type = driver.find_element_by_xpath(industry_xpath).text.splitlines(0)[1]
    industries.append(industry_type)
    
    x = x+1

In [186]:
# to check if we have all information
print(len(job_id))
print(len(post_date))
print(len(company_name))
print(len(post_title))
print(len(job_location))
print(len(job_desc))
print(len(level))
print(len(emp_type))
print(len(functions))
print(len(industries))

23
23
23
23
23
23
23
23
23
23


In [187]:
# creating a dataframe
job_data = pd.DataFrame({'Job ID': job_id,
'Date': post_date,
'Company Name': company_name,
'Post': post_title,
'Location': job_location,
'Description': job_desc,
'Level': level,
'Type': emp_type,
'Function': functions,
'Industry': industries
})

# cleaning description column
job_data['Description'] = job_data['Description'].str.replace('\n',' ')

print(job_data.info())
job_data.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 23 entries, 0 to 22
Data columns (total 10 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   Job ID        23 non-null     object
 1   Date          23 non-null     object
 2   Company Name  23 non-null     object
 3   Post          23 non-null     object
 4   Location      23 non-null     object
 5   Description   23 non-null     object
 6   Level         23 non-null     object
 7   Type          23 non-null     object
 8   Function      23 non-null     object
 9   Industry      23 non-null     object
dtypes: object(10)
memory usage: 1.9+ KB
None


,Job ID,Date,Company Name,Post,Location,Description,Level,Type,Function,Industry
0,2401687696,2021-02-07,Dyson,Digital Optimisation Analyst,서울,About Us Relentlessly solving problems is at ...,신입,풀타임,광고디자인마케팅,IT컴퓨터 소프트웨어소매
1,2421699533,2021-02-22,Viva Republica (Toss),[토스증권] Data Analyst,"서울, 대한민국",Viva Republica (Toss)에서 채용공고 게시자에게 직접 메시지를 보냅니...,과장·부장,풀타임,IT애널리스트전략·기획,금융 서비스IT은행
2,2412547747,2021-02-16,Han Sung Motor 한성자동차,Data Analyst,"서울, 대한민국",Han Sung Motor 한성자동차에서 채용공고 게시자에게 직접 메시지를 보냅니다...,신입,풀타임,IT,자동차
3,2433340415,2021-02-21,Morningstar,Associate data research analyst - Seoul,서울,An associate data research analyst will perfor...,대리,풀타임,연구애널리스트IT,경영 자문금융 서비스투자 관리
4,2383871476,2021-01-28,Asurion,Data Analyst,서울 인천 지역,"To apply the position, please attached your de...",대리,풀타임,애널리스트IT,IT보험


In [188]:
job_data.to_csv("linkedin_job_crawling_0223.csv", index = False)

In [202]:
df.head()

,Job ID,Date,Company Name,Post,Location,Description,Level,Type,Function,Industry
0,2401687696,2021-02-07,Dyson,Digital Optimisation Analyst,서울,About Us Relentlessly solving problems is at ...,신입,풀타임,광고디자인마케팅,IT컴퓨터 소프트웨어소매
1,2421699533,2021-02-22,Viva Republica (Toss),[토스증권] Data Analyst,"서울, 대한민국",Viva Republica (Toss)에서 채용공고 게시자에게 직접 메시지를 보냅니...,과장·부장,풀타임,IT애널리스트전략·기획,금융 서비스IT은행
2,2412547747,2021-02-16,Han Sung Motor 한성자동차,Data Analyst,"서울, 대한민국",Han Sung Motor 한성자동차에서 채용공고 게시자에게 직접 메시지를 보냅니다...,신입,풀타임,IT,자동차
3,2433340415,2021-02-21,Morningstar,Associate data research analyst - Seoul,서울,An associate data research analyst will perfor...,대리,풀타임,연구애널리스트IT,경영 자문금융 서비스투자 관리
4,2383871476,2021-01-28,Asurion,Data Analyst,서울 인천 지역,"To apply the position, please attached your de...",대리,풀타임,애널리스트IT,IT보험


In [203]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 23 entries, 0 to 22
Data columns (total 10 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   Job ID        23 non-null     int64         
 1   Date          23 non-null     datetime64[ns]
 2   Company Name  23 non-null     object        
 3   Post          23 non-null     object        
 4   Location      23 non-null     object        
 5   Description   23 non-null     object        
 6   Level         23 non-null     object        
 7   Type          23 non-null     object        
 8   Function      23 non-null     object        
 9   Industry      23 non-null     object        
dtypes: datetime64[ns](1), int64(1), object(8)
memory usage: 1.9+ KB


In [201]:
df["Date"] = pd.to_datetime(df["Date"], format = "%Y/%m/%d")

In [204]:
df["Location"].unique()

array(['서울', '서울, 대한민국', '서울 인천 지역', '강남구', '경기도, 대한민국'], dtype=object)

In [211]:
df = df.sort_values(by = "Date", ascending = False)

In [213]:
df.reset_index(drop = True, inplace = True)

In [219]:
df = df.replace("L&#39;Oréal", "L'Oreal Korea")
df.to_csv("linkedin_job_crawling_0223.csv", index = False)